In [9]:
import inference.InferCode_Inference as II
import numpy as np

In [12]:
token_dict_path = "/home/stanley/Desktop/dictionaries/v2_dic/token2id.json"
type_dict_path = "/home/stanley/Desktop/dictionaries/v2_dic/type2id.json"
model_weight_path = "/home/stanley/Desktop/SSPTM-neg_b32/epoch_195.pkl"
subtree_count = 633916 # 633916 for the SS-PTM v2
dimension = 64

In [14]:
model = II.InferCode_Inference(model_weight_path, token_dict_path, type_dict_path, subtree_count, dimension)
code1 = "/home/stanley/Desktop/code1.xml"
code2 = "/home/stanley/Desktop/code2.xml"
code3 = "/home/stanley/Desktop/code3.xml"

def distance(vec1, vec2):
    return np.linalg.norm(vec1-vec2)

def cos(x, y):
    return np.dot(x,y) / (np.sqrt(np.dot(x,x)) * np.sqrt(np.dot(y,y)))

def dot(x, y):
    return np.dot(x, y)

vec_1 = model.code2vec(code1)
vec_2 = model.code2vec(code2)
vec_3 = model.code2vec(code3)

d12 = distance(vec_1, vec_2)
d13 = distance(vec_1, vec_3)
d23 = distance(vec_2, vec_3)

print(vec_3)

print("distance between function 1 and 2", d12)
print("distance between function 1 and 3", d13)
print("distance between function 2 and 3", d23)

[ 0.4165447   0.02106206  0.21456537 -0.20890737  0.01887543 -0.28900337
 -0.16506582  0.06129588  0.11038661  0.33295074 -0.18611464  0.09937148
 -0.11224072  0.08472121 -0.00870761 -0.05764532 -0.13722184 -0.00628154
 -0.06020184  0.11528009 -0.26011023 -0.11481699  0.36679396 -0.06340963
  0.3024111  -0.14350587  0.12406252  0.03655527 -0.17112517  0.22023389
  0.12368517 -0.16442573  0.19359732  0.04715182  0.16680512  0.00842091
  0.23412004 -0.10194509  0.160095    0.48644257 -0.10088089 -0.05501895
 -0.22460425 -0.10902257 -0.2385906   0.02267981 -0.08752649  0.18430728
 -0.06254062  0.10973315  0.11760657 -0.03797961  0.1989398  -0.1274761
  0.11316322  0.02855133  0.06627183  0.16992337 -0.1384761   0.0443601
  0.08751611  0.12251977 -0.17169917 -0.31807715]
distance between function 1 and 2 0.3921932
distance between function 1 and 3 0.8215564
distance between function 2 and 3 0.78649956
